# FP Growth algorithm

### Importing pandas as dependency to read csv and the dataframe functions

In [1]:
import pandas as pd

## Class FP-node
    This represents the note of the tree
    parameters are self explainatory

In [2]:
class fp_node:
    def __init__(self, name, count, parent):
        self.name = name
        # during mining process, frequency of a transaction can be more than one
        self.count = count
        self.nodeLink = None
        # dictionary of children, key are the name, value is the fp-node object
        self.children = {}
        # parent is required during mining to select the paths ending with some element
        self.parent = parent

    def inc(self, val):
        self.count += val

    # display tree in text. Useful for debugging
    def disp(self, ind=1):
        print('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

### fp_growth prepares the tree and mines it

In [3]:
class fp_growth():

    def __init__(self, dataset, minsup):
        self.raw_data = dataset
        self.dataset = dataset
        if minsup < 1:
            self.minsup = (int)(len(dataset)*minsup)
        else:
            self.minsup = minsup

    #since the input is one hot encoded, this will transform it to list of tags
    def reformat(self):

        def get_tags(row):
            tags = set()
            for c in self.dataset.columns:
                if int(row[c]) == 1:
                    tags.add(c)
            return tags

        dataset_list = list(self.raw_data.apply(get_tags, axis=1))

        retDict = {}
        for trans in dataset_list:
            retDict[frozenset(trans)] = 1
        self.dataset = retDict

    # create FP-tree from dataset but don't mine
    def createTree(self):  
        freq_table = {}

        # first pass counts frequency of occurance
        for transaction in self.dataset:  
            for item in transaction:
                freq_table[item] = freq_table.get(
                    item, 0) + self.dataset[transaction]

        tempdict = {}
        # remove items not meeting minSup
        for key in freq_table:  
            if freq_table[key] >= self.minsup:
                tempdict[key] = freq_table[key]

        freq_table = tempdict
        freqItemSet = set(freq_table.keys())

        # if no items meet min support -->get out
        if len(freqItemSet) == 0:
            return None, None  
        headerTable = {}
        for k in freq_table:
            # reformat headerTable to use Node link
            headerTable[k] = [freq_table[k], None]
        headerTable = {k: v for k, v in sorted(
            headerTable.items(), key=lambda p: p[1][0])}
        # create root of the tree
        root = fp_node('Null Set', 1, None)  
        # go through dataset 2nd time
        for transaction, count in self.dataset.items():  
            localD = {}
            # put transaction items in order
            for item in transaction:  
                if item in freqItemSet:
                    localD[item] = freq_table[item]
            if len(localD) > 0:
                localD = {k: v for k, v in sorted(localD.items())}
                orderedItems = [k for k, v in sorted(
                    localD.items(), key=lambda p: p[1], reverse=True)]
                # populate tree with ordered freq itemset
                self.updateTree(orderedItems, root, headerTable, count)
        return root, headerTable  

    def updateTree(self, items, root, headerTable, count):
        # check if orderedItems[0] in retTree.children
        if items[0] in root.children:  
            # update frequency
            root.children[items[0]].inc(count)  
        else:  
            root.children[items[0]] = fp_node(items[0], count, root)
            # update header table
            if headerTable[items[0]][1] == None:  
                headerTable[items[0]][1] = root.children[items[0]]
            else:
                self.updateHeader(
                    headerTable[items[0]][1], root.children[items[0]])
        # call updateTree() with remaining ordered items
        if len(items) > 1:  
            self.updateTree(
                items[1::], root.children[items[0]], headerTable, count)

    def updateHeader(self, nodeToTest, targetNode):  
        while (nodeToTest.nodeLink != None):  
            nodeToTest = nodeToTest.nodeLink
        nodeToTest.nodeLink = targetNode

    # ascends from leaf node to root
    def ascendTree(self, leafNode, prefixPath):  
        if leafNode.parent != None:
            prefixPath.append(leafNode.name)
            self.ascendTree(leafNode.parent, prefixPath)

    # treeNode comes from header table
    def findPrefixPath(self, treeNode):  
        condPats = {}
        while treeNode != None:
            prefixPath = []
            self.ascendTree(treeNode, prefixPath)
            if len(prefixPath) > 1:
                condPats[frozenset(prefixPath[1:])] = treeNode.count
            treeNode = treeNode.nodeLink
        return condPats

    # mine the tree for patterns
    def mine(self, root, headerTable, prefixbase=tuple()):
        #return if no root or children
        if root == None or len(root.children) == 0:
            return prefixbase
        mined_freq_patterns = set()

        for key in headerTable:
            # if support is > minsup
            if headerTable[key][0] >= self.minsup:
                # add prefix till now + this key into result
                mined_freq_patterns.add(prefixbase+(key,))
                # find all transactions ending at this key
                prefix_paths = self.findPrefixPath(headerTable[key][1])
                # create fp tree with the above transactions
                temp = fp_growth(prefix_paths, self.minsup)
                myFPtree, myHeaderTab = temp.createTree()
                # if tree is created, mine it
                if myFPtree != None:
                    mined_pattern = temp.mine(myFPtree, myHeaderTab,
                                              prefixbase+(key,))
                    # add these mined results to final results set
                    mined_freq_patterns = mined_freq_patterns.union(
                        mined_pattern)
        return mined_freq_patterns


In [4]:
df = pd.read_csv('../Data/Processed/encoded.csv',index_col=False)
df = df.drop(columns=['Unnamed: 0'])
df.head()

,district,age,highest_qualification,no_of_dwelling_rooms,household_have_electricity_Yes,is_water_filter_Yes,rural_Rural,rural_Urban,sex_Female,sex_Male,...,cooking_fuel_Electricity,cooking_fuel_Firewood,cooking_fuel_Kerosene,cooking_fuel_LPG/PNG,cooking_fuel_No cooking,kitchen_availability_Cooking inside the house:Has kitchen,kitchen_availability_Cooking outside house:Has kitchen,kitchen_availability_Does not have kitchen,kitchen_availability_No cooking,status_of_living
0,MUZAFFARPUR,28.0,5.0,2.0,1,0,1,0,0,1,...,0,1,0,0,0,1,0,0,0,0.952696
1,MUZAFFARPUR,25.0,5.0,2.0,1,0,1,0,1,0,...,0,1,0,0,0,1,0,0,0,0.952696
2,MUZAFFARPUR,24.0,5.0,2.0,1,0,1,0,0,1,...,0,1,0,0,0,1,0,0,0,0.952696
3,MUZAFFARPUR,20.0,5.0,2.0,1,0,1,0,0,1,...,0,1,0,0,0,1,0,0,0,0.952696
4,MUZAFFARPUR,25.0,4.0,2.0,1,0,1,0,0,1,...,0,1,0,0,0,1,0,0,0,0.952696


### Create age groups old, man, child

In [5]:
df["is_old"] = 0
df["is_child"] = 0
df["is_man"] = 0
df.loc[df['age'] <=10, ['is_child']] = 1
df.loc[df['age'] >40, ['is_old']] = 1
df.loc[df['age'] >10, ['is_man']] = 1
df.loc[df['age'] >41, ['is_man']] = 0

### Create income groups based on status of living

In [6]:
one_third = df['status_of_living'].quantile(0.3)
two_third = df['status_of_living'].quantile(0.9)
max_status = df['status_of_living'].max()
min_status = 0.0
print(one_third,two_third,max_status)

0.4325331344687818 2.7856903219454203 8.784504245624463


In [7]:
df["low_income"] = 0
df["mid_income"] = 0
df["high_income"] = 0
df.loc[df['status_of_living'] <=one_third, ['low_income']] = 1
df.loc[df['status_of_living'] >two_third, ['high_income']] = 1
df.loc[(df['status_of_living'] >one_third) & (df['status_of_living'] <=two_third), ['mid_income']] = 1

In [8]:
df.head()

,district,age,highest_qualification,no_of_dwelling_rooms,household_have_electricity_Yes,is_water_filter_Yes,rural_Rural,rural_Urban,sex_Female,sex_Male,...,kitchen_availability_Cooking outside house:Has kitchen,kitchen_availability_Does not have kitchen,kitchen_availability_No cooking,status_of_living,is_old,is_child,is_man,low_income,mid_income,high_income
0,MUZAFFARPUR,28.0,5.0,2.0,1,0,1,0,0,1,...,0,0,0,0.952696,0,0,1,0,1,0
1,MUZAFFARPUR,25.0,5.0,2.0,1,0,1,0,1,0,...,0,0,0,0.952696,0,0,1,0,1,0
2,MUZAFFARPUR,24.0,5.0,2.0,1,0,1,0,0,1,...,0,0,0,0.952696,0,0,1,0,1,0
3,MUZAFFARPUR,20.0,5.0,2.0,1,0,1,0,0,1,...,0,0,0,0.952696,0,0,1,0,1,0
4,MUZAFFARPUR,25.0,4.0,2.0,1,0,1,0,0,1,...,0,0,0,0.952696,0,0,1,0,1,0


### Select binary attributes. Eliminate scalar and strings

In [9]:
df1 = df[["household_have_electricity_Yes", "is_water_filter_Yes", "sex_Female", "sex_Male", "is_telephone_No", "occupation_status_Agricultural Wage labourer", "occupation_status_Attended educational Institution", "occupation_status_Attending routine domestic chores etc.", "occupation_status_Beggars", "occupation_status_Cultivator", "occupation_status_Did not work but was seeking and/or available for work", "occupation_status_Non-Agricultural Wage labourer.", "occupation_status_Not able to work due to disability", "occupation_status_Others", "occupation_status_Prostitutes/Sex Workers", "occupation_status_Regular salaried/wage employees", "occupation_status_Rentiers;Pensioners;Other remittance recipients", "occupation_status_Self-Employed(excluding cultivators)-Employers", "occupation_status_Self-Employed(excluding cultivators)-Un-paid family labourer", "occupation_status_Self-Employed(excluding cultivators)Own Account Workers", "occupation_status_Too old to work", "illness_type_Acute respiratory infection", "illness_type_Diarrhoea", "illness_type_Dysentery", "illness_type_Fever of short duration with rashes", "illness_type_Fever with chill/rigors (Malaria etc)", "illness_type_Jaundice with fever", "illness_type_No Illness", "illness_type_Other types of fever", "illness_type_Others", "illness_type_Reproductive Tract Infection (RTI)", "symptoms_pertaining_illness_Asymptomatic", "symptoms_pertaining_illness_Diseases of cardiovascular system", "symptoms_pertaining_illness_Diseases of gastrointestinal system", "symptoms_pertaining_illness_Diseases of genito urinary system", "symptoms_pertaining_illness_Diseases of musculo-skeletal system", "symptoms_pertaining_illness_Diseases of respiratory system", "symptoms_pertaining_illness_Diseases of the central nervous system", "symptoms_pertaining_illness_ENT problems/diseases", "symptoms_pertaining_illness_Elephantiasis", "symptoms_pertaining_illness_Eye Problem/diseases", "symptoms_pertaining_illness_Goitre", "symptoms_pertaining_illness_Mouth and Dental Problems", "symptoms_pertaining_illness_No Symptoms of chronic diseases", "symptoms_pertaining_illness_Others", "symptoms_pertaining_illness_Skin diseases", "diagnosed_for_Anaemia", "diagnosed_for_Asthma/chronic respiratory failure", "diagnosed_for_Blood cancer/leukemia", "diagnosed_for_Cancer-Breast", "diagnosed_for_Cancer-Gastro-intestinal system", "diagnosed_for_Cancer-Genitourinary system", "diagnosed_for_Cancer-Respiratory system", "diagnosed_for_Cataract", "diagnosed_for_Chronic heart disease", "diagnosed_for_Chronic heart disease/failure", "diagnosed_for_Chronic liver disease", "diagnosed_for_Chronic liver failure", "diagnosed_for_Chronic renal failure", "diagnosed_for_Chronic skin diseases - psoriasis", "diagnosed_for_Diabetes", "diagnosed_for_Epilepsy", "diagnosed_for_Flourosis", "diagnosed_for_Gall stone /cholecystitis", "diagnosed_for_Glaucoma", "diagnosed_for_Goitre/ Thyroid disorder", "diagnosed_for_Hypertension", "diagnosed_for_Leprosy", "diagnosed_for_Myocardial infarction/heart attack", "diagnosed_for_Not diagnosed", "diagnosed_for_Others(Hernia;Hydrocele;Peptic Ulcer etc)", "diagnosed_for_Piles/Anal fissure & Anal fistula", "diagnosed_for_Pyorrhea", "diagnosed_for_Renal stone", "diagnosed_for_Rheumatic fever/Rheumatic heart disease", "diagnosed_for_Rheumatoid arthritis / osteoarthritis", "diagnosed_for_Sinusitis,Tonsilitis", "diagnosed_for_Skin Cancer", "diagnosed_for_Stroke/Cerebrovascular accident", "diagnosed_for_Tuberculosis", "diagnosed_for_Tumour(any type)", "house_status_New House", "house_status_Non-residential", "house_status_Vacant", "house_structure_Kuccha", "house_structure_Others", "house_structure_Pucca", "house_structure_Semi Pucca", "owner_status_Others", "owner_status_Rented", "drinking_water_source_Hand pump", "drinking_water_source_Piped water into dwelling/yard/plot", "drinking_water_source_Protected dug well", "drinking_water_source_Public tap/standpipe", "drinking_water_source_Surface water", "drinking_water_source_Tanker /truck/Cart with Surface watersmall tank", "drinking_water_source_Tube well or Borehole", "drinking_water_source_Unprotected dug well", "drinking_water_source_other sources", "toilet_used_Flush/Pour flush latrine connected:-To piped sewer system", "toilet_used_Open pit /Pit latrine without slab", "toilet_used_Pit latrine with slab", "toilet_used_Pit latrine(without flush/ pour flush):-Ventilated Improved Pit (VIP)", "toilet_used_To pit latrine", "toilet_used_To septic tank", "toilet_used_To somewhere else", "toilet_used_community toilet", "toilet_used_open defecation(field; brush;jungle etc.)", "toilet_used_service latrine", "lighting_source_Any other", "lighting_source_Electricity", "lighting_source_Kerosene", "lighting_source_No lighting", "lighting_source_Other Oils", "lighting_source_Solar", "cooking_fuel_Any other", "cooking_fuel_Biogas", "cooking_fuel_Coal/lignite/Charcoal", "cooking_fuel_Cow dung cake", "cooking_fuel_Crop Residue", "cooking_fuel_Electricity", "cooking_fuel_Firewood", "cooking_fuel_Kerosene", "cooking_fuel_LPG/PNG", "cooking_fuel_No cooking", "kitchen_availability_Cooking inside the house:Has kitchen", "kitchen_availability_Cooking outside house:Has kitchen", "kitchen_availability_Does not have kitchen", "kitchen_availability_No cooking", "is_old", "is_child", "is_man", "low_income", "mid_income", "high_income"]]

### split the data district wise

In [10]:
districts = ['PATNA', 'MUZAFFARPUR', 'SITAMARHI', 'ROHTAS', 'GAYA', 'PURNIA']
district_data = {}
for district_name in districts:
    district_data[district_name] = df1.loc[df['district'] == district_name]

### create tree

In [11]:
mining_session = fp_growth(district_data['GAYA'], 0.1)
mining_session.reformat()
myFPtree, myHeaderTab = mining_session.createTree()
print("FP_tree created")

FP_tree created


### display tree

In [ ]:
myFPtree.disp()

### Mine tree

In [ ]:
mining_session.mine(myFPtree, myHeaderTab)

### find the confidence (a->b)

In [ ]:
def find_conf(temp_data,lhs,rhs):
    a = len(temp_data[(temp_data[lhs] == 1)])
    b = len(temp_data[(temp_data[lhs] == 1) & (temp_data[rhs] == 1)])
    c = len(temp_data[(temp_data[rhs] == 1)])
    print("LHS->RHS:",b/a)
    print("RHS->LHS:",b/c)


In [ ]:
# find_conf('toilet_used_Flush/Pour_flush_latrine_connected:-To_piped_sewer_system','symptoms_pertaining_illness_No_Symptoms_of_chronic_diseases')
find_conf(df1,'toilet_used_Flush/Pour flush latrine connected:-To piped sewer system','illness_type_No Illness')